# Lösungsvorschlag für Aufgabe 6

In [1]:
using LinearAlgebra

#### A) Das Wissen über die Cholesky-Zerlegung aus dem Beweis von [B, Satz 8.3]

Das Programm aus der Vorlesung, d.h.

In [2]:
function CholeskyVL(A) 
    m,~ = size(A)
    L = zeros(eltype(A),m,m)
    for k = 1:m
        lk = L[1:k-1,1:k-1]\A[1:k-1,k]
        L[k,1:k-1] = lk'
        L[k,k] = sqrt(A[k,k] - lk'*lk)
    end
    return L
end;

basiert auf der Partitionierung
$$
A_k = 
\begin{pmatrix}
A_{k-1} & a_k \\
a'_k    & \alpha_k 
\end{pmatrix},\quad
L_k = 
\begin{pmatrix}
L_{k-1} &  \\
l'_k    & \lambda_k 
\end{pmatrix},\quad
L'_k = 
\begin{pmatrix}
L'_{k-1} & l_k \\
  & \lambda_k 
\end{pmatrix}.
$$

Weiterhin ist bekannt, dass die Lösung $x_k$ des linearen Hilfssystems $L'_{k-1}x_k=-l_k$ die Beziehung
$$
\tau_k := \alpha_k - l'_kl_k =
\begin{pmatrix}
x_k\\ 
1
\end{pmatrix}'
A_k
\begin{pmatrix}
x_k\\ 
1
\end{pmatrix}
$$
erfüllt. Der Beweis von [B, Satz 8.3] zeigt nun: 

> Ist $A$ (und damit alle Hauptuntermatrizen $A_k$) s.p.d., dann ist $\tau_k>0$ für $k=1:m$. 

#### B) Beweis der Umkehrung
Umgekehrt gilt (für _selbstadjungiertes_ $A$ ist der Term $\tau_k$ auch im Fall $\mathbb{K} = \mathbb{C}$ immer _reell_): 

> Ist $A$ selbstadjungiert, aber _nicht_ positiv definit, dann muss es ein $k$ mit $\tau_k\le 0$
geben. 

Denn wäre dem nicht so, dann würde der Cholesky-Algorithmus mit $\lambda_k = \sqrt{\tau_k}>0$ für alle $k$ durchlaufen und man hätte nach Konstruktion am Schluss $A=LL'$, so dass $A$ nach [B, Lemma 9.1] tatsächlich s.p.d sein müsste.

#### C) Zusammenfassung

Also gilt:  

> Ein selbstadjungiertes $A$ ist genau dann _nicht_ s.p.d., wenn für ein $k$ gilt $\tau_k \le 0$. 

Für dieses $k$ hat man dann mit dem oben definierten Vektor $x_k$ alle in der Aufgabe gesuchten Informationen: für den daraus konstruierten Vektor

$$
x = 
\begin{pmatrix}
x_k\\ 
1\\
0
\end{pmatrix}\quad\text{gilt nämlich}\quad
x'Ax = 
\begin{pmatrix}
x_k\\ 
1
\end{pmatrix}'
A_k
\begin{pmatrix}
x_k\\ 
1
\end{pmatrix} = \tau_k  \le 0.
$$

#### D) Programm

Die Beobachtung aus (c) führt zu folgender einfachen Modifikation des Programms:

In [3]:
function CholeskyTest(A) 
    m,~ = size(A)
    L = zeros(eltype(A),m,m)
    for k = 1:m
        lk = L[1:k-1,1:k-1]\A[1:k-1,k]
        L[k,1:k-1] = lk'
        # --------- Beginn der Modifikation ---------
        if (τₖ = real(A[k,k] - dot(lk,lk))) > 0
            L[k,k] = sqrt(τₖ)
        else
            @warn "A ist nicht positiv definit"
            x = zeros(eltype(A),m); x[1:k-1] = -L[1:k-1,1:k-1]'\lk; x[k] = 1;            
            return L, x # Ausgabe von L enthält evtl. wertvolle Information
        end  
        # --------- Ende der Modifikation ---------
    end
    return L, nothing
end;

#### E) Ergänzung: Test

Ein Test des Programms war zwar in der Aufgabe nicht gefordert, sollte aber grundsätzlich immer Bestandteil einer _Eigenkorrektur_ sein. Vorschlag: 

In [6]:
# ruhig mehrfach wiederholen: wo steht jeweils die 1 in x...

m = 10
A = randn(m,m)
A = (A+A')/2 + (sqrt(2m)-2)I # selbstadjungiert, aber mit sehr hoher W-keit nicht p.d.
# A = A + 3I # mit sehr hoher W-keit s.p.d.

L, x = CholeskyTest(A);

if x == nothing
    println("A ist positiv definit")
else
    println(">>> x≠0 mit x'Ax = $(dot(x,A*x)) ≤ 0 gefunden:")
    println("x = ",x)
end

┌ Warning: A ist nicht positiv definit
└ @ Main In[3]:11


>>> x≠0 mit x'Ax = -0.050889982626297015 ≤ 0 gefunden:
x = [-0.5511158041801816, 1.8507897329350114, 0.6142373389987632, 0.1666243242025426, -1.5822208658695711, -1.1161188460352587, -0.02501552727512851, 0.8381448139938666, 1.0, 0.0]
